In [1]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12,49,51,56,91,126,145,163,165,166,167,168,169,171,173,174,176,177,242,244,246,248,252,255,274,290,291,292,294,295,296,332,344,366,374,376,397,414,440,491,620,634,639,642,643,645,710,713,760,769,810,829,929,954,979,1001,1002,1003,1004,1005,1024,1037,1041,1043,1062,1086,1099,1100,1121,1129,1136,1152,1153,1166,1168,1182,1193,1204,1205,1207,1208,1216,1226,1228,1230,1232,1234) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (11,13,48,50,55,90,125,144,162,163,164,166,168,169,170,171,172,174,176,241,243,245,247,251,254,273,289,291,292,293,295,296,312,313,314,331,343,365,373,375,396,413,439,490,619,623,630,633,638,642,644,669,712,759,768,788,809,828,928,953,978,1000,1001,1002,1003,1004,1023,1036,1040,1042,

In [2]:
train.head()

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
0,0,3,32,3.0,NaN,323011,3854,481,1975,1,...,99.0,NaN,99,NaN,99,NaN,99,NaN,99,NaN
1,1,2,26,NaN,8.0,268131,2441,344,1981,1,...,NaN,NaN,1,NaN,2,NaN,2,NaN,2,NaN
2,2,1,16,NaN,7.0,167581,754,143,1995,1,...,1.0,NaN,2,NaN,2,NaN,2,NaN,2,NaN
3,3,4,44,5.0,NaN,445071,5705,604,1980,1,...,NaN,NaN,2,NaN,2,NaN,99,NaN,99,NaN
4,4,4,43,NaN,6.0,436161,5645,592,1958,1,...,NaN,NaN,1,NaN,1,NaN,1,NaN,1,NaN


indicative feature
['DG5_9', 'DG6', 'DL0', 'DL1', 'DL2', 'DL4_22', 'G2P1_11', 'MT1A', 'MT17_6', 'MT18_4', 'MT18_5', 'FF14_3', 'FF14_4', 'FF14_5', 'FF14_6', 'MM38_14', 'MM40_14', 'FL3', 'GN2', 'GN3', 'GN5', 'DG1', 'DG8a', 'DG9a', 'FF7_5']
'MM38_14'
rate('GN2', 2)
rate('FF14_6') 1
rate('FF14_5')  1
rate('FF14_4')  1
rate('FF14_3') 1
rate('MT18_5') 1
rate('MT18_4') 1
rate('GN5') 1 2
decide later
rate('MT17_6') 

In [33]:
train["GN5"].unique()

array([ nan,   0.,   2.,   1.,   4.])

In [50]:
# function
# train['FF7_5']
def rate(s):
    for value in range(len(train[s].unique())):
        if len(train[train[s] == value]) != 0:
            print (value, sum(train[train[s] == value]["is_female"])/len(train[train[s] == value]))
    
rate('GN5') 


1 0.2699154828268297
2 0.8202099737532809
3 0.5922350922350922
4 0.4862053369516056


In [27]:
# drop the nas by "any" or "all"
t_nona = train.dropna(axis = 1, how = "any")
t_nona.head()

,train_id,AA3,AA4,AA7,AA14,AA15,DG1,is_female,DG3,DG3A,...,LN1A,LN1B,LN2_1,LN2_2,LN2_3,LN2_4,GN2,GN3,GN4,GN5
0,0,3,32,323011,3854,481,1975,1,3,4,...,1,1,1,1,1,1,99,99,99,99
1,1,2,26,268131,2441,344,1981,1,8,4,...,3,3,1,1,3,4,1,2,2,2
2,2,1,16,167581,754,143,1995,1,3,2,...,4,4,1,1,2,2,2,2,2,2
3,3,4,44,445071,5705,604,1980,1,3,4,...,3,2,1,1,4,5,2,2,99,99
4,4,4,43,436161,5645,592,1958,1,3,4,...,2,1,2,4,4,4,1,1,1,1


# delete columns with over 80 percent NA

In [77]:
# function delete columns with over 80 percent NA
def delete80na_col():
    over80 = []
    other = []
    for col in list(train.columns.values):
        per = train[col].isnull().sum()/len(train[col])
        if per > 0.4:
#             print (">80")
            over80.append(col)
        else:
            other.append(col)
    return over80, other
delete80na_col()
            
# # train['FF14_4'].isnull().sum()/len(train['AA3'])       
# for col in list(train.columns.values):
#     print (col)
over80, other = delete80na_col()
over80

['AA5',
 'DG3A_OTHERS',
 'DG4_OTHERS',
 'DG9b',
 'DG9c',
 'DG10b',
 'DG10c',
 'DG11b',
 'DG11c',
 'DG12B_1',
 'DG12B_2',
 'DG12C_1',
 'DG12C_2',
 'DG13_1',
 'DG13_2',
 'DG13_3',
 'DG13_4',
 'DG13_5',
 'DG13_96',
 'DG13_7',
 'DG13_OTHERS',
 'DG14',
 'DG14_OTHERS',
 'DL1_OTHERS',
 'DL2',
 'DL2_23_OTHERS',
 'DL2_96_OTHERS',
 'DL3',
 'DL4_OTHERS',
 'DL7',
 'DL8',
 'DL12',
 'DL12_OTHERS',
 'DL13',
 'DL27',
 'DL28',
 'DL28_OTHERS',
 'G2P1_1',
 'G2P1_2',
 'G2P1_3',
 'G2P1_4',
 'G2P1_5',
 'G2P1_6',
 'G2P1_7',
 'G2P1_8',
 'G2P1_9',
 'G2P1_10',
 'G2P1_11',
 'G2P1_12',
 'G2P1_13',
 'G2P1_14',
 'G2P1_15',
 'G2P1_16',
 'G2P1_96',
 'G2P1_99',
 'G2P1_OTHERS',
 'G2P2_1',
 'G2P2_2',
 'G2P2_3',
 'G2P2_4',
 'G2P2_5',
 'G2P2_6',
 'G2P2_7',
 'G2P2_8',
 'G2P2_9',
 'G2P2_10',
 'G2P2_11',
 'G2P2_12',
 'G2P2_13',
 'G2P2_14',
 'G2P2_15',
 'G2P2_16',
 'G2P2_96',
 'G2P2_1_OTHERS',
 'G2P2_2_OTHERS',
 'G2P2_4_OTHERS',
 'G2P2_6_OTHERS',
 'G2P2_7_OTHERS',
 'G2P2_8_OTHERS',
 'G2P2_9_OTHERS',
 'G2P2_10_OTHERS',
 'G2P2_

In [80]:
train_delete_over80 = train.drop(over80, axis = 1)

# delete columns with text

In [ ]:
lst = []
acc = 0
for i in (train.dtypes):
    if str(i) == "object":
        lst.append(acc)
    acc = acc+1
    
lst2 = []
for i in lst:
    lst2.append(train.dtypes.keys()[i])

In [82]:
X = train_delete_over80.drop(["is_female","train_id"], axis = 1)
Y = train_delete_over80.is_female

In [83]:
test = test.reindex(columns = X.columns, fill_value = 0)

In [84]:
from sklearn.model_selection import train_test_split

In [85]:
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.4, random_state=42)

In [86]:
from sklearn.ensemble import RandomForestClassifier
TARGET = "is_female"

In [87]:
Ntree = 500
clf = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
clf.fit(X_train, y_train)

ValueError: could not convert string to float: 'Telugu'

In [19]:
y_pred = clf.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm)

[[2955  440]
 [ 401 3506]]


In [20]:
y_pred

array([0, 0, 0, ..., 1, 1, 1])

In [21]:
y_prob = clf.predict_proba(X_test)

In [22]:
y_prob

array([[ 0.894,  0.106],
       [ 0.696,  0.304],
       [ 0.55 ,  0.45 ],
       ..., 
       [ 0.404,  0.596],
       [ 0.468,  0.532],
       [ 0.11 ,  0.89 ]])

In [23]:
from sklearn import metrics

In [24]:
metrics.roc_auc_score(y_test, y_prob[:,1])

0.95500063516523526

In [25]:
# feature selection
from sklearn.feature_selection import SelectKBest

In [73]:
train[["is_female","MM28"]][:50]
a = train[["is_female","MM31_9"]].loc[train["MM31_9"] == 2.0]
# a.groupby(["MM31_9","is_female"]).value
# a.pivot(index='is_female', columns='MM31_9')
sum(train["is_female"].loc[train["MM31_9"] == 2.0])
len(train["is_female"].loc[train["MM31_9"] == 2.0])

64

In [74]:
X.columns

Index(['AA3', 'AA4', 'AA7', 'AA14', 'AA15', 'DG1', 'DG3', 'DG3A', 'DG4',
       'DG5_1',
       ...
       'LN1A', 'LN1B', 'LN2_1', 'LN2_2', 'LN2_3', 'LN2_4', 'GN2', 'GN3', 'GN4',
       'GN5'],
      dtype='object', length=237)

In [77]:
dic ={}
for feature in zip(X.columns, clf.feature_importances_):
    dic[feature[1]] = feature[0]
sorted(dic.keys())

[0.0,
 2.5308194883818775e-06,
 2.8699922725177057e-06,
 7.7351872073187215e-06,
 9.0457861986758171e-06,
 1.0696567464831093e-05,
 1.1487894791318577e-05,
 1.2154996715974923e-05,
 1.2157281351453624e-05,
 1.3775475974014467e-05,
 1.4368418196152514e-05,
 1.4770180298541072e-05,
 1.8426274781264229e-05,
 1.9450911752970867e-05,
 2.3538961248779397e-05,
 2.4867387551159507e-05,
 2.5598050578228686e-05,
 2.7164529920667378e-05,
 3.2722399162158611e-05,
 3.7451479316742632e-05,
 3.9813480768029011e-05,
 4.1023970347847128e-05,
 4.4705681176936932e-05,
 4.5119014672365827e-05,
 5.02161016285769e-05,
 5.1234798943489909e-05,
 5.4350105813450348e-05,
 5.7963242735490312e-05,
 5.8838588549038493e-05,
 6.2870704630045256e-05,
 6.7792425365157842e-05,
 7.0140194292703917e-05,
 7.180208699541169e-05,
 7.2477467996036947e-05,
 7.7568975843250797e-05,
 8.033077388119964e-05,
 8.0999134108799452e-05,
 9.2041024639323061e-05,
 9.4660920303399763e-05,
 0.00010408703901697832,
 0.00011077250602151041

In [78]:
dic[0.11546906914365694]

'DL0'

In [117]:
lst = []
for i in sorted(dic.keys()):
    if i>0.01:
        lst.append(dic[i])
lst

['DL14',
 'DG5_4',
 'AA15',
 'AA14',
 'DG8a',
 'AA7',
 'DG4',
 'GN2',
 'DG1',
 'MT10',
 'GN3',
 'GN5',
 'MT2',
 'GN4',
 'DG3',
 'FL4',
 'DL1',
 'DG6',
 'DL0']

In [82]:
X_new = train[lst]
Y_new = train.is_female
# test_new = test.reindex[lst]

x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)

Ntree = 500
clf = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
clf.fit(x_n_train, y_n_train)

y_pred = clf.predict(x_n_test)

cm = confusion_matrix(y_n_test, y_pred)
print (cm)

y_prob = clf.predict_proba(x_n_test)
metrics.roc_auc_score(y_n_test, y_prob[:,1])

test_n = test.reindex(columns = x_n_test.columns)

y_pred_f1 = clf.predict(test_n)
y_prob = clf.predict_proba(test_n)


In [90]:
y_prob

array([[ 0.004,  0.996],
       [ 0.85 ,  0.15 ],
       [ 0.01 ,  0.99 ],
       ..., 
       [ 0.998,  0.002],
       [ 0.468,  0.532],
       [ 0.24 ,  0.76 ]])

In [91]:
import csv


In [99]:
with open("submit.csv",'w') as f:
    fieldnames = ["test_id","is_female"]
    wri = csv.DictWriter(f, delimiter=',',fieldnames= fieldnames)
    acc = 0
    wri.writeheader()
    for i in y_prob:
        wri.writerow({"test_id":acc, "is_female": i[1]})
        acc = acc+1

In [100]:
t = pd.read_csv("submit.csv")

In [120]:
new_feature = pd.read_csv("new_feature.csv")
new_feature = new_feature["Column Name"].tolist()
# new_feature  

In [122]:
lst = lst + new_feature


In [124]:
newlist = lst

In [127]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values="NaN", strategy = "mean", axis = 1)
imp = imp.fit(train[newlist])

In [ ]:
X_new = imp.transform(train[newlist])
Y_new = train.is_female
# test_new = test.reindex[lst]

x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)

Ntree = 500
clf = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
clf.fit(x_n_train, y_n_train)

y_pred = clf.predict(x_n_test)

cm = confusion_matrix(y_n_test, y_pred)
print (cm)

y_prob = clf.predict_proba(x_n_test)
print(metrics.roc_auc_score(y_n_test, y_prob[:,1]))

In [135]:
test_n = test.reindex(columns = newlist)
imp = imp.fit(test)
test = imp.transform(test)


y_pred_f1 = clf.predict(test_n)
y_prob = clf.predict_proba(test_n)

ValueError: could not convert string to float: 'My child'